# 🔴 Gören Göz - POTHOLE (Colab + Kaggle)

Kaggle'dan pothole dataseti indirip eğitiyoruz.

In [ ]:
!nvidia-smi
!pip install ultralytics kaggle -q
print("✅ Kurulum tamam")

## 1️⃣ Kaggle API

1. [kaggle.com/settings](https://www.kaggle.com/settings) → Create New Token
2. İnen kaggle.json'daki username ve key'i aşağıya yaz

In [ ]:
import os
os.makedirs('/root/.kaggle', exist_ok=True)

# ⚠️ KAGGLE BİLGİLERİNİ GİR!
KAGGLE_USERNAME = "YOUR_USERNAME"  # <-- Değiştir!
KAGGLE_KEY = "YOUR_KEY"  # <-- Değiştir!

with open('/root/.kaggle/kaggle.json', 'w') as f:
    f.write(f'{{"username": "{KAGGLE_USERNAME}", "key": "{KAGGLE_KEY}"}}')
!chmod 600 /root/.kaggle/kaggle.json
print("✅ Kaggle API hazır")

## 2️⃣ Dataset İndir

In [ ]:
# Pothole dataset indir
print("📥 Pothole indiriliyor...")
!kaggle datasets download -d chitholian/potholes-detection-yolov8 -p /content --unzip
print("✅ İndirildi!")

In [ ]:
# Yapıyı kontrol et
print("📁 İndirilen dosyalar:")
!ls -la /content/
print("\n📁 Detay:")
!find /content -name "*.yaml" -o -name "*.txt" | head -20

In [ ]:
# Dataset yolunu bul
from pathlib import Path
import os

# Olası yolları kontrol et
possible_paths = [
    "/content",
    "/content/potholes-detection-yolov8",
    "/content/Potholes-Detection-YOLOv8",
]

DATASET_DIR = None
for p in possible_paths:
    if os.path.exists(f"{p}/train/images") or os.path.exists(f"{p}/train"):
        DATASET_DIR = p
        break

# Yoksa bul
if not DATASET_DIR:
    for item in Path("/content").iterdir():
        if item.is_dir() and (item / "train").exists():
            DATASET_DIR = str(item)
            break

print(f"📁 Dataset: {DATASET_DIR}")
!ls {DATASET_DIR}

In [ ]:
# Train ve Valid yapısını bul
import yaml

# Olası yapıları kontrol et
train_img = None
val_img = None

structures = [
    ("train/images", "valid/images"),
    ("train", "valid"),
    ("images/train", "images/val"),
]

for t, v in structures:
    if os.path.exists(f"{DATASET_DIR}/{t}"):
        train_img = t
        val_img = v if os.path.exists(f"{DATASET_DIR}/{v}") else t.replace("train", "test")
        break

print(f"Train: {train_img}")
print(f"Valid: {val_img}")

In [ ]:
# data.yaml oluştur
data = {
    "path": DATASET_DIR,
    "train": train_img,
    "val": val_img,
    "nc": 1,
    "names": ["pothole"]
}

with open('/content/data.yaml', 'w') as f:
    yaml.dump(data, f)

print("✅ data.yaml oluşturuldu:")
print(yaml.dump(data))

In [ ]:
# Sayıları kontrol
train_n = len(list(Path(f"{DATASET_DIR}/{train_img}").glob("*")))
val_n = len(list(Path(f"{DATASET_DIR}/{val_img}").glob("*"))) if os.path.exists(f"{DATASET_DIR}/{val_img}") else 0
print(f"📊 Train: {train_n}, Valid: {val_n}")

## 3️⃣ Training

In [ ]:
from ultralytics import YOLO

model = YOLO('yolo11n.pt')
print(f"🚀 Training: {train_n} train, {val_n} val")

results = model.train(
    data='/content/data.yaml',
    epochs=100,
    imgsz=640,
    batch=16,
    device=0,
    patience=20,
    project='goren_goz',
    name='pothole_v1',
    exist_ok=True
)

In [ ]:
from IPython.display import Image, display
display(Image(filename='goren_goz/pothole_v1/results.png', width=800))

In [ ]:
best = YOLO('goren_goz/pothole_v1/weights/best.pt')
m = best.val()
print(f"\n📊 mAP50: {m.box.map50*100:.1f}%")

In [ ]:
from google.colab import files
files.download('goren_goz/pothole_v1/weights/best.pt')
print("✅ Model indirildi!")
print("📁 Kaydet: backend/models/yolo11n_pothole.pt")